# Kubeflow Pipelines with Katib component

In this notebook you will:
- Create Katib Experiment using random algorithm.
- Use median stopping rule as an early stopping algorithm.
- Use Kubernetes Job with pytorch mnist training container as a Trial template.
- Create Pipeline to get the optimal hyperparameters.

Reference documentation:
- https://kubeflow.org/docs/components/katib/experiment/#random-search
- https://kubeflow.org/docs/components/katib/early-stopping/
- https://kubeflow.org/docs/pipelines/overview/concepts/component/

**Note**: This Pipeline runs in the multi-user mode. Follow [this guide](https://github.com/kubeflow/katib/tree/master/examples/v1beta1/kubeflow-pipelines#multi-user-pipelines-setup) to give your Notebook access to Kubeflow Pipelines.

In [1]:
# Install required packages (Kubeflow Pipelines and Katib SDK).
# !pip install kfp==1.8.12
!pip install kubeflow-katib==0.13.0

  Using cached kubeflow_katib-0.13.0-py3-none-any.whl.metadata (1.4 kB)
Using cached kubeflow_katib-0.13.0-py3-none-any.whl (89 kB)


In [1]:
import kfp
import kfp.dsl as dsl
from kfp import components

from kubeflow.katib import ApiClient
from kubeflow.katib import V1beta1ExperimentSpec
from kubeflow.katib import V1beta1AlgorithmSpec
from kubeflow.katib import V1beta1EarlyStoppingSpec
from kubeflow.katib import V1beta1EarlyStoppingSetting
from kubeflow.katib import V1beta1ObjectiveSpec
from kubeflow.katib import V1beta1ParameterSpec
from kubeflow.katib import V1beta1FeasibleSpace
from kubeflow.katib import V1beta1TrialTemplate
from kubeflow.katib import V1beta1TrialParameterSpec

/tmp/ipykernel_159/642187433.py:1: FutureWarning: KFP will drop support for Python 3.8 on Oct 1, 2024. To use new versions of the KFP SDK after that date, you will need to upgrade to Python >= 3.9. See https://devguide.python.org/versions/ for more details.
  import kfp


## Define an Experiment

You have to create an Experiment object before deploying it. This Experiment is similar to [this](https://github.com/kubeflow/katib/blob/master/examples/v1beta1/early-stopping/median-stop.yaml) YAML.

In [2]:
# Experiment name and namespace.
experiment_name = "median-stop-2"
experiment_namespace = "kubeflow-user-example-com"

# Trial count specification.
max_trial_count = 18
max_failed_trial_count = 3
parallel_trial_count = 2

# Objective specification.
objective=V1beta1ObjectiveSpec(
    type="minimize",
    goal= 0.001,
    objective_metric_name="loss",
)

# Algorithm specification.
algorithm=V1beta1AlgorithmSpec(
    algorithm_name="random",
)

# Early Stopping specification.
early_stopping=V1beta1EarlyStoppingSpec(
    algorithm_name="medianstop",
    algorithm_settings=[
        V1beta1EarlyStoppingSetting(
            name="min_trials_required",
            value="2"
        )
    ]
)

# Experiment search space.
# In this example we tune learning rate, number of layer and optimizer.
# Learning rate has bad feasible space to show more early stopped Trials.
parameters=[
    V1beta1ParameterSpec(
        name="lr",
        parameter_type="double",
        feasible_space=V1beta1FeasibleSpace(
            min="0.01",
            max="0.3"
        ),
    ),
    V1beta1ParameterSpec(
        name="momentum",
        parameter_type="double",
        feasible_space=V1beta1FeasibleSpace(
            min="0.5",
            max="0.9"
        ),
    ),
]


## Define a Trial template

In this example, the Trial's Worker is the Kubernetes Job.

In [3]:
# JSON template specification for the Trial's Worker Kubernetes Job.
trial_spec={
    "apiVersion": "batch/v1",
    "kind": "Job",
    "spec": {
        "template": {
            "metadata": {
                "labels": {
                     "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "training-container",
                        "image": "docker.io/kubeflowkatib/pytorch-mnist-cpu:v0.14.0",
                        "command": [
                            "python3",
                            "/opt/pytorch-mnist/mnist.py",
                            "--epochs=1",
                            "--batch-size=16",
                            "--lr=${trialParameters.learningRate}",
                            "--momentum=${trialParameters.momentum}",
                        ]
                    }
                ],
                "restartPolicy": "Never"
            }
        }
    }
}

# Configure parameters for the Trial template.
# We set the retain parameter to "True" to not clean-up the Trial Job's Kubernetes Pods.
trial_template=V1beta1TrialTemplate(
    retain=True,
    primary_container_name="training-container",
    trial_parameters=[
        V1beta1TrialParameterSpec(
            name="learningRate",
            description="Learning rate for the training model",
            reference="lr"
        ),
        V1beta1TrialParameterSpec(
            name="momentum",
            description="Momentum for the training model",
            reference="momentum"
        ),
    ],
    trial_spec=trial_spec
)

## Define an Experiment specification

Create an Experiment specification from the above parameters.

In [4]:
experiment_spec=V1beta1ExperimentSpec(
    max_trial_count=max_trial_count,
    max_failed_trial_count=max_failed_trial_count,
    parallel_trial_count=parallel_trial_count,
    objective=objective,
    algorithm=algorithm,
    early_stopping=early_stopping,
    parameters=parameters,
    trial_template=trial_template
)

# Create a Pipeline using Katib component

The best hyperparameters are printed after Experiment is finished.
The Experiment is not deleted after the Pipeline is finished.

In [9]:
from kfp import dsl, components
from kfp.dsl import pipeline, component
# from kubernetes.client import ApiClient

# # Load Katib launcher component
katib_experiment_launcher_op = components.load_component_from_url(
    "https://raw.githubusercontent.com/kubeflow/pipelines/master/components/contrib/kubeflow/katib-launcher/component.yaml")

@component
def print_best_hp(best_hp: dict):
    print(f"Best HyperParameters: {best_hp}")

@dsl.pipeline(
    name="Launch Katib early stopping Experiment",
    description="An example to launch Katib Experiment with early stopping"
)
def median_stop_pipeline():
    # Katib launcher component
    op = katib_experiment_launcher_op(
        experiment_name=experiment_name,
        experiment_namespace=experiment_namespace,
        experiment_spec=ApiClient().sanitize_for_serialization(experiment_spec),
        experiment_timeout_minutes=60,
        delete_finished_experiment=False,
    )

    # Replace ContainerOp with a component
    # print_best_hp(best_hp=op.outputs)


In [10]:
# # Get the Katib launcher.
# # katib_experiment_launcher_op = components.load_component_from_url(
# #     "https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/katib-launcher/component.yaml")
# katib_experiment_launcher_op = components.load_component_from_url(
#     "https://raw.githubusercontent.com/kubeflow/pipelines/master/components/contrib/kubeflow/katib-launcher/component.yaml")

# @dsl.pipeline(
#     name="Launch Katib early stopping Experiment",
#     description="An example to launch Katib Experiment with early stopping"
# )

# def median_stop():

#     # Katib launcher component.
#     # Experiment Spec should be serialized to a valid Kubernetes object.
#     op = katib_experiment_launcher_op(
#         experiment_name=experiment_name,
#         experiment_namespace=experiment_namespace,
#         experiment_spec=ApiClient().sanitize_for_serialization(experiment_spec),
#         experiment_timeout_minutes=60,
#         delete_finished_experiment=False)

#     # Output container to print the results.
#     op_out = dsl.ContainerOp(
#         name="best-hp",
#         image="library/bash:4.4.23",
#         command=["sh", "-c"],
#         arguments=["echo Best HyperParameters: %s" % op.output],
#     )

# Run the Kubeflow Pipeline

You can check the Katib Experiment info in the Katib UI.

In [11]:
# # Run the Kubeflow Pipeline in the user's namespace.
# kfp.Client().create_run_from_pipeline_func(median_stop,  namespace=experiment_namespace, arguments={})

In [12]:
# Run the Kubeflow Pipeline in the user's namespace.
kfp.Client().create_run_from_pipeline_func(median_stop_pipeline,  namespace=experiment_namespace, arguments={})

RunPipelineResult(run_id=5b857074-eebe-46df-8245-17aed6ab3c6b)